In [2]:
import pandas as pd
import ast

In [5]:
chennai_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\chennai_cars.xlsx")

In [6]:
chennai_df.head()

,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'Minivans', 'k...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Low...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
4,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...


In [7]:
chennai_df['new_car_detail'].head()

0    {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...
1    {'it': 0, 'ft': 'Petrol', 'bt': 'Minivans', 'k...
2    {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...
3    {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
4    {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
Name: new_car_detail, dtype: object

In [8]:
# Extract data from each structured column
car_details_df = pd.json_normalize(chennai_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

In [9]:
car_details_df.columns

Index(['it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 'oem',
       'model', 'modelYear', 'centralVariantId', 'variantName', 'price',
       'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl',
       'trendingText.heading', 'trendingText.desc'],
      dtype='object')

In [10]:
# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = chennai_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Display the extracted DataFrame
print(car_overview_df.columns)

Index(['Registration Year_value', 'Registration Year_icon',
       'Insurance Validity_value', 'Insurance Validity_icon',
       'Fuel Type_value', 'Fuel Type_icon', 'Seats_value', 'Seats_icon',
       'Kms Driven_value', 'Kms Driven_icon', 'RTO_value', 'RTO_icon',
       'Ownership_value', 'Ownership_icon', 'Engine Displacement_value',
       'Engine Displacement_icon', 'Transmission_value', 'Transmission_icon',
       'Year of Manufacture_value', 'Year of Manufacture_icon'],
      dtype='object')


In [11]:
# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = chennai_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Display the extracted DataFrame
print(car_features_df.columns)

Index(['Top_Features', 'Common_Icon'], dtype='object')


In [12]:
# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# **top_specs unpacks all key-value pairs from the top_specs dictionary.
# **detailed_specs unpacks all key-value pairs from the detailed_specs dictionary.

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = chennai_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Display the extracted DataFrame
print(car_specs_df.columns)

Index(['Engine', 'Max Power', 'Torque', 'Wheel Size', 'Seats', 'Color',
       'Engine Type', 'Displacement', 'Max Torque', 'No of Cylinder',
       'Values per Cylinder', 'Fuel Suppy System', 'Turbo Charger', 'Length',
       'Width', 'Height', 'Wheel Base', 'Kerb Weight', 'Gear Box',
       'Drive Type', 'Seating Capacity', 'Steering Type', 'Front Brake Type',
       'Rear Brake Type', 'Tyre Type', 'Alloy Wheel Size', 'No Door Numbers',
       'Cargo Volumn', 'Common_Icon', 'Mileage', 'Value Configuration',
       'Compression Ratio', 'Super Charger', 'Front Tread', 'Rear Tread',
       'Gross Weight', 'Turning Radius', 'Top Speed', 'Acceleration',
       'BoreX Stroke', 'Ground Clearance Unladen'],
      dtype='object')


In [13]:
car_details_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,SUV,"20,000",Automatic,1,1st Owner,Kia,Kia Sonet,2022,8654,Turbo DCT Anniversary Edition,₹ 11.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Minivans,"20,687",Manual,1,1st Owner,Maruti,Maruti Eeco,2015,4025,7 Seater Standard BSIV,₹ 4.15 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,SUV,"30,000",Manual,1,1st Owner,Nissan,Nissan Magnite,2021,8135,Turbo XV Premium BSVI,₹ 7.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"59,247",Manual,1,1st Owner,Hyundai,Hyundai i10,2015,1579,Sportz 1.1L,₹ 3.98 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"50,000",Manual,1,1st Owner,Honda,Honda Jazz,2015,1341,1.2 VX i VTEC,₹ 5.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,0,Petrol,Hatchback,"42,891",Automatic,1,1st Owner,Maruti,Maruti Swift,2018,6190,AMT VXI,₹ 6.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1415,0,Diesel,SUV,"59,100",Manual,1,1st Owner,Renault,Renault Duster,2018,5050,85PS Diesel RxZ,₹ 8.90 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1416,0,Petrol,SUV,"55,913",Automatic,1,1st Owner,Tata,Tata Nexon,2018,6364,1.2 Revotron XZA Plus,₹ 8.01 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1417,0,Diesel,SUV,"65,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Tiguan,2017,5849,2.0 TDI Highline,₹ 20.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


In [14]:
# Create a DataFrame from the car_links
car_links_df = chennai_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
chennai_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

# Add a new column named 'city' with the value 'chennai'
chennai_combined_df.insert(0, 'city', 'chennai')

# Save the final combined DataFrame to a CSV file
output_file = 'chennai_cars_final.csv'
chennai_combined_df.to_csv(output_file, index=False)

print(f"Structured data saved to {output_file}")

Structured data saved to chennai_cars_final.csv


In [15]:
chennai_combined_df

,city,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Super Charger,Front Tread,Rear Tread,Gross Weight,Turning Radius,Top Speed,Acceleration,BoreX Stroke,Ground Clearance Unladen,car_links
0,chennai,0,Petrol,SUV,"20,000",Automatic,1,1st Owner,Kia,Kia Sonet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,chennai,0,Petrol,Minivans,"20,687",Manual,1,1st Owner,Maruti,Maruti Eeco,...,No,1280mm,1290mm,1540kg,4.5 metres,145 Kmph,15.7 Seconds,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
2,chennai,0,Petrol,SUV,"30,000",Manual,1,1st Owner,Nissan,Nissan Magnite,...,No,NaN,NaN,NaN,5.0,NaN,11.7,72.2 x 81.3,NaN,https://www.cardekho.com/used-car-details/used...
3,chennai,0,Petrol,Hatchback,"59,247",Manual,1,1st Owner,Hyundai,Hyundai i10,...,No,1400mm,1385mm,NaN,4.7 metres,165 Kmph,14.3 Seconds,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
4,chennai,0,Petrol,Hatchback,"50,000",Manual,1,1st Owner,Honda,Honda Jazz,...,No,NaN,NaN,NaN,5.1 meters,172 Kmph,13.7 Seconds,NaN,NaN,https://www.cardekho.com/used-car-details/used...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,chennai,0,Petrol,Hatchback,"42,891",Automatic,1,1st Owner,Maruti,Maruti Swift,...,No,1530mm,1530mm,1315kg,4.8 Meters,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1415,chennai,0,Diesel,SUV,"59,100",Manual,1,1st Owner,Renault,Renault Duster,...,No,1560mm,1567mm,1777kg,5.2 metres,156 Kmph,13.9 Seconds,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1416,chennai,0,Petrol,SUV,"55,913",Automatic,1,1st Owner,Tata,Tata Nexon,...,No,1540 mm,1530 mm,NaN,5.1m,154.19 kmph,NaN,77x85.8,209 mm,https://www.cardekho.com/used-car-details/used...
1417,chennai,0,Diesel,SUV,"65,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Tiguan,...,No,1578mm,1568mm,2250kg,5.75meters,NaN,NaN,74.5 x 81 mm,NaN,https://www.cardekho.com/used-car-details/used...


In [17]:
Bangalore_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\bangalore_cars.xlsx")
Bangalore_df.shape

(1481, 5)

In [18]:
# Extract data from each structured column
car_details_df = pd.json_normalize(Bangalore_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = Bangalore_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = Bangalore_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = Bangalore_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Create a DataFrame from the car_links
car_links_df = Bangalore_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
Bangalore_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

Bangalore_combined_df.insert(0, 'city', 'Bangalore')

# Save the final combined DataFrame to a CSV file
Bangalore_combined_df.to_csv(r'Bangalore_cars_final.csv')

In [19]:
Bangalore_combined_df.head()

,city,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Common_Icon,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen,car_links
0,Bangalore,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,...,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,Bangalore,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,...,NaN,NaN,"Tubeless,Radial",4,352-litres,,16,16,NaN,https://www.cardekho.com/buy-used-car-details/...
2,Bangalore,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,...,150 kmph,14.3 Seconds,Tubeless,5,242-litres,,14,14,NaN,https://www.cardekho.com/used-car-details/used...
3,Bangalore,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,...,172km/hr,14.2 Seconds,"Tubeless,Radial",4,407-litres,,14,14,NaN,https://www.cardekho.com/buy-used-car-details/...
4,Bangalore,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,...,190 Kmph,12 Seconds,"Tubeless,Radial",5,353-litres,,16,16,NaN,https://www.cardekho.com/used-car-details/used...


In [20]:
Bangalore_combined_df.shape

(1481, 84)

In [21]:
Delhi_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\delhi_cars.xlsx")
Delhi_df.shape

(1485, 5)

In [22]:
# Extract data from each structured column
car_details_df = pd.json_normalize(Delhi_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = Delhi_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = Delhi_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = Delhi_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Create a DataFrame from the car_links
car_links_df = Delhi_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
Delhi_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

Delhi_combined_df.insert(0, 'city', 'Delhi')

# Save the final combined DataFrame to a CSV file
Delhi_combined_df.to_csv(r'Delhi_cars_final.csv')

In [23]:

Delhi_combined_df.head()

,city,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Turning Radius,Top Speed,Acceleration,Gross Weight,Front Tread,Rear Tread,BoreX Stroke,Compression Ratio,Ground Clearance Unladen,car_links
0,Delhi,0,Diesel,SUV,"10,000",Automatic,1,1st Owner,Kia,Kia Seltos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,Delhi,0,Petrol,SUV,"57,437",Manual,2,2nd Owner,Hyundai,Hyundai Creta,...,5.3 metres,165 Kmph,10.5 Seconds,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
2,Delhi,0,Petrol,SUV,"8,000",Automatic,1,1st Owner,Mercedes-Benz,Mercedes-Benz GLC,...,NaN,217 Kmph,NaN,2360kg,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
3,Delhi,0,Petrol,Hatchback,"28,151",Manual,2,2nd Owner,Maruti,Maruti Swift,...,4.8,NaN,NaN,1335,1530,1530,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
4,Delhi,0,Petrol,SUV,"60,000",Manual,1,1st Owner,Hyundai,Hyundai Creta,...,5.3 metres,165 Kmph,10.5 Seconds,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...


In [24]:
Delhi_combined_df.shape

(1485, 84)

In [25]:
Hyderabad_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\hyderabad_cars.xlsx")
Hyderabad_df.shape

(1483, 5)

In [26]:
# Extract data from each structured column
car_details_df = pd.json_normalize(Hyderabad_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = Hyderabad_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = Hyderabad_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = Hyderabad_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Create a DataFrame from the car_links
car_links_df = Hyderabad_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
Hyderabad_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

Hyderabad_combined_df.insert(0, 'city', 'Hyderabad')

# Save the final combined DataFrame to a CSV file
Hyderabad_combined_df.to_csv(r'Hyderabad_cars_final.csv')
Hyderabad_combined_df.shape


(1483, 84)

In [27]:
Kolkata_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\kolkata_cars.xlsx")
Kolkata_df.shape

(1381, 5)

In [28]:
# Extract data from each structured column
car_details_df = pd.json_normalize(Kolkata_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = Kolkata_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = Kolkata_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = Kolkata_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Create a DataFrame from the car_links
car_links_df = Kolkata_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
Kolkata_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

Kolkata_combined_df.insert(0, 'city', 'Kolkata')

# Save the final combined DataFrame to a CSV file
Kolkata_combined_df.to_csv(r'Kolkata_cars_final.csv')
Kolkata_combined_df.shape

(1381, 84)

In [29]:
Kolkata_combined_df.head()

,city,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Tyre Type,Alloy Wheel Size,No Door Numbers,Common_Icon,Ground Clearance Unladen,Cargo Volumn,Compression Ratio,Acceleration,Top Speed,car_links
0,Kolkata,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,...,"Tubeless,Radial",17,4,,NaN,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,Kolkata,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,...,NaN,NaN,5,,185mm,222,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
2,Kolkata,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,...,"Tubeless, Radial",NaN,5,,NaN,405,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
3,Kolkata,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,...,"Tubeless,Radial",14,5,,NaN,295 Lit,:1,12.96 Sec,NaN,https://www.cardekho.com/buy-used-car-details/...
4,Kolkata,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,...,"Tubeless,Radial",16,5,,NaN,460-litres,NaN,8.2 Seconds,212 Kmph,https://www.cardekho.com/used-car-details/used...


In [31]:
Jaipur_df = pd.read_excel(r"C:\Users\jaguh\OneDrive\Desktop\car dekho2\jaipur_cars.xlsx")
Jaipur_df.shape

(1120, 5)

In [32]:
# Extract data from each structured column
car_details_df = pd.json_normalize(Jaipur_df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)) 

# Function to extract 'top' items from the car overview, including icons
def extract_overview_data(overview):
    if isinstance(overview, str):
        overview = ast.literal_eval(overview)
    
    # Extract key, value, and icon
    extracted_data = {}
    for item in overview.get('top', []):
        key = item['key']
        extracted_data[f"{key}_value"] = item['value']
        extracted_data[f"{key}_icon"] = item.get('icon', '')  # Use empty string if icon is missing
    
    return extracted_data

# Apply the extraction function to the 'new_car_overview' column
car_overview_df = Jaipur_df['new_car_overview'].apply(extract_overview_data).apply(pd.Series)

# Function to extract features from the car feature data
def extract_features(features):
    if isinstance(features, str):
        features = ast.literal_eval(features)
    
    top_features = [item['value'] for item in features.get('top', [])]
    common_icon = features.get('commonIcon', '')
    
    return top_features, common_icon

# Apply the function and create a DataFrame
car_features_df = Jaipur_df['new_car_feature'].apply(extract_features).apply(pd.Series)
car_features_df.columns = ['Top_Features', 'Common_Icon']

# Function to extract specifications from a structured dictionary
def extract_specifications(specifications):
    if isinstance(specifications, str):
        specifications = ast.literal_eval(specifications)
        
    top_specs = {item['key']: item['value'] for item in specifications.get('top', [])}
    
    detailed_specs = {}
    for category in specifications.get('data', []):
        for item in category.get('list', []):
            detailed_specs[item['key']] = item['value']
    
    return {**top_specs, **detailed_specs, 'Common_Icon': specifications.get('commonIcon', '')}

# Apply the extraction function to the 'new_car_specifications' column
car_specs_df = Jaipur_df['new_car_specs'].apply(extract_specifications).apply(pd.Series)

# Create a DataFrame from the car_links
car_links_df = Jaipur_df['car_links'].reset_index(drop=True)

# Combine all DataFrames into one
Jaipur_combined_df = pd.concat([
    car_details_df.reset_index(drop=True),
    car_overview_df.reset_index(drop=True),
    car_features_df.reset_index(drop=True),
    car_specs_df.reset_index(drop=True),
    car_links_df], axis=1)

Jaipur_combined_df.insert(0, 'city', 'Jaipur')

# Save the final combined DataFrame to a CSV file
Jaipur_combined_df.to_csv(r'Jaipur_cars_final.csv')
Jaipur_combined_df.shape


(1120, 84)

In [33]:
Jaipur_combined_df.head()

,city,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Common_Icon,Gross Weight,BoreX Stroke,Compression Ratio,Ground Clearance Unladen,car_links
0,Jaipur,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,...,Tubeless,16,5,295-litres,,NaN,NaN,NaN,NaN,https://www.cardekho.com/used-car-details/used...
1,Jaipur,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,...,"Radial, Tubeless",NaN,5,268,,1335,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
2,Jaipur,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,...,"Tubeless, Radial",NaN,5,235-litres,,1250kg,73 X 82 mm,11.0:1,NaN,https://www.cardekho.com/used-car-details/used...
3,Jaipur,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,...,Tubeless,NaN,5,256,,NaN,NaN,NaN,NaN,https://www.cardekho.com/buy-used-car-details/...
4,Jaipur,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,...,Tubeless Tyres,NaN,5,180-liters,,1350kg,69 x 72 mm,NaN,NaN,https://www.cardekho.com/used-car-details/used...


In [34]:
# Reset the index for each DataFrame to ensure unique index
chennai_combined_df = chennai_combined_df.reset_index(drop=True)
Bangalore_combined_df = Bangalore_combined_df.reset_index(drop=True)
Delhi_combined_df = Delhi_combined_df.reset_index(drop=True)
Hyderabad_combined_df = Hyderabad_combined_df.reset_index(drop=True)
Jaipur_combined_df = Jaipur_combined_df.reset_index(drop=True)
Kolkata_combined_df = Kolkata_combined_df.reset_index(drop=True)

# Ensure that all DataFrames have the same columns
common_columns = list(set(chennai_combined_df.columns).intersection(
    Bangalore_combined_df.columns,
    Delhi_combined_df.columns,
    Hyderabad_combined_df.columns,
    Jaipur_combined_df.columns,
    Kolkata_combined_df.columns
))

# Filter each DataFrame to include only the common columns
chennai_combined_df = chennai_combined_df[common_columns]
Bangalore_combined_df = Bangalore_combined_df[common_columns]
Delhi_combined_df = Delhi_combined_df[common_columns]
Hyderabad_combined_df = Hyderabad_combined_df[common_columns]
Jaipur_combined_df = Jaipur_combined_df[common_columns]
Kolkata_combined_df = Kolkata_combined_df[common_columns]

# Concatenate all city DataFrames
all_city_cars_df = pd.concat([chennai_combined_df, Bangalore_combined_df, Delhi_combined_df, Hyderabad_combined_df,
                              Jaipur_combined_df, Kolkata_combined_df], ignore_index=True)

# Save the combined DataFrame to a CSV file
all_city_cars_df.to_csv('all_city_cars.csv', index=False)

print("Data for all cities saved to 'all_city_cars.csv'")

Data for all cities saved to 'all_city_cars.csv'


In [35]:
all_city_cars_df.shape

(8369, 84)